In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [32]:
ImT1_L = cv2.imread('./Data/00/image_0/000000.png', 0)    #0 flag returns a grayscale image
ImT1_R = cv2.imread('./Data/00/image_1/000000.png', 0)

ImT2_L = cv2.imread('./Data/00/image_0/000001.png', 0)
ImT2_R = cv2.imread('./Data/00/image_1/000001.png', 0)

# cv2.imshow('ImT1_L', ImT1_L)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

block = 15

#emperical values from P1, P2 as suggested in Ocv documentation
P1 = 0 #block * block * 8
P2 = 0 #block * block * 32

disparityEngine = cv2.StereoSGBM_create(minDisparity=0,numDisparities=16, blockSize=block, P1=P1, P2=P2)
ImT1_disparity = disparityEngine.compute(ImT1_L, ImT1_R).astype(np.float32)
#cv2.imwrite('disparity.png', ImT1_disparity)
ImT1_disparityA = np.divide(ImT1_disparity, 16.0)

ImT2_disparity = disparityEngine.compute(ImT2_L, ImT2_R).astype(np.float32)
ImT2_disparityA = np.divide(ImT2_disparity, 16.0)

In [33]:
TILE_H = 10
TILE_W = 20
fastFeatureEngine = cv2.FastFeatureDetector_create()


# keypoints = fastFeatureEngine.detect(ImT1_L)
# ftDebug = ImT1_L
# ftDebug = cv2.drawKeypoints(ImT1_L, keypoints, ftDebug, color=(255,0,0))
# cv2.imwrite('ftDebug.png', ftDebug)

#20x10 (wxh) tiles for extracting less features from images 
H,W = ImT1_L.shape
kp = []
idx = 0
for y in range(0, H, TILE_H):
    for x in range(0, W, TILE_W):
        imPatch = ImT1_L[y:y+TILE_H, x:x+TILE_W]
        keypoints = fastFeatureEngine.detect(imPatch)
        for pt in keypoints:
            pt.pt = (pt.pt[0] + x, pt.pt[1] + y)
        
        if (len(keypoints) > 10):
            keypoints = sorted(keypoints, key=lambda x: -x.response)
            for kpt in keypoints[0:10]:
                kp.append(kpt)
        else:
            for kpt in keypoints:
                kp.append(kpt)

ftDebug = ImT1_L
ftDebug = cv2.drawKeypoints(ImT1_L, kp, ftDebug, color=(255,0,0))
cv2.imwrite('ftDebug.png', ftDebug)

True

In [96]:
# pack keypoint 2-d coords into numpy array
trackPoints1 = np.zeros((len(kp),1,2), dtype=np.float32)
for i,kpt in enumerate(kp):
    trackPoints1[i,:,0] = kpt.pt[0]
    trackPoints1[i,:,1] = kpt.pt[1]

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 3,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 50, 0.03))

trackPoints2, st, err = cv2.calcOpticalFlowPyrLK(ImT1_L, ImT2_L, trackPoints1, None, flags=cv2.MOTION_AFFINE, **lk_params)

# separate points that were tracked successfully
ptTrackable = np.where(st == 1, 1,0).astype(bool)
trackPoints1_KLT = trackPoints1[ptTrackable, ...]
trackPoints2_KLT_t = trackPoints2[ptTrackable, ...]
trackPoints2_KLT = np.around(trackPoints2_KLT_t)

# among tracked points take points within error measue
error = 4
errTrackablePoints = err[ptTrackable, ...]
errThresholdedPoints = np.where(errTrackablePoints < error, 1, 0).astype(bool)
trackPoints1_KLT = trackPoints1_KLT[errThresholdedPoints, ...]
trackPoints2_KLT = trackPoints2_KLT[errThresholdedPoints, ...]

In [98]:
#compute right image disparity displaced points
trackPoints1_KLT_L = trackPoints1_KLT
trackPoints2_KLT_L = trackPoints2_KLT

trackPoints1_KLT_R = np.copy(trackPoints1_KLT_L)
trackPoints2_KLT_R = np.copy(trackPoints2_KLT_L)
selectedPointMap = np.zeros(trackPoints1_KLT_L.shape[0])

disparityMinThres = 0.0
disparityMaxThres = 100.0
for i in range(trackPoints1_KLT_L.shape[0]):
    T1Disparity = ImT1_disparityA[int(trackPoints1_KLT_L[i,1]), int(trackPoints1_KLT_L[i,0])]
    T2Disparity = ImT2_disparityA[int(trackPoints2_KLT_L[i,1]), int(trackPoints2_KLT_L[i,0])]
    
    if (T1Disparity > disparityMinThres and T1Disparity < disparityMaxThres 
        and T2Disparity > disparityMinThres and T2Disparity < disparityMaxThres):
        trackPoints1_KLT_R[i, 0] = trackPoints1_KLT_L[i, 0] - T1Disparity
        trackPoints2_KLT_R[i, 0] = trackPoints2_KLT_L[i, 0] - T2Disparity
        selectedPointMap[i] = 1
        
selectedPointMap = selectedPointMap.astype(bool)
trackPoints1_KLT_L_3d = trackPoints1_KLT_L[selectedPointMap, ...]
trackPoints1_KLT_R_3d = trackPoints1_KLT_R[selectedPointMap, ...]
trackPoints2_KLT_L_3d = trackPoints2_KLT_L[selectedPointMap, ...]
trackPoints2_KLT_R_3d = trackPoints2_KLT_R[selectedPointMap, ...]



477.0

In [28]:
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                 maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 1000, 0.01))

p0 = cv2.goodFeaturesToTrack(ImT1_L, mask = None, **feature_params)
p1, st, err = cv2.calcOpticalFlowPyrLK(ImT1_L, ImT2_L, p0, None, **lk_params)
